In [1]:
#!pip3 install pandas
import pandas as pd
import Group5FileLib as fl

# Unzip files if they are not unzipped on local machine
fl.UnzipFiles()

# Combine Zipped Files
viewFile = fl.CombineFiles()

# Load Lot Fact File
lotFile = fl.LoadLotFacts()

File exists data/VCardTransactions-NoName.csv - NOT un-zipping
File exists data/VCardTransactions-NoName2.csv - NOT un-zipping
File exists data/vtrans.csv - NOT merging


In [2]:
# Read in csv file
vtrans = pd.read_csv(viewFile)
vtrans.head()

,tTransID,FullName,CardNumber,LotNumber,EffectiveGroupName,EntranceTime,ExitTime,Sort_Time,BusinessHours,NWHours,UseMoreThan3Hours,HourExceed3Hours,NoEntry,noexit,Consecutive,Overnight
0,22840709,NaN,12777,13,NaN,2022-07-11 12:29:09.493,NaN,2022-07-11 12:29:09.493,NaN,NaN,NaN,NaN,0,1,0,0
1,22840716,NaN,47951,40,NaN,2022-07-11 12:29:09.367,NaN,2022-07-11 12:29:09.367,NaN,NaN,NaN,NaN,0,1,0,0
2,22840719,NaN,58760,3,NaN,2022-07-11 12:29:04.227,NaN,2022-07-11 12:29:04.227,NaN,NaN,NaN,NaN,0,1,0,0
3,22840717,NaN,57369,20,NaN,2022-07-11 12:29:02.680,NaN,2022-07-11 12:29:02.680,NaN,NaN,NaN,NaN,0,1,0,0
4,22840710,NaN,13947,11,NaN,2022-07-11 12:28:53.320,NaN,2022-07-11 12:28:53.320,NaN,NaN,NaN,NaN,0,1,0,0


In [ ]:
#!pip3 install numpy
import numpy as np

# Clean vtrans
data = pd.DataFrame()
vtrans['EntranceTime'] = pd.to_datetime(vtrans['EntranceTime'], format='%Y/%m/%d')
vtrans['date'] = vtrans['EntranceTime'].dt.strftime('%Y/%m/%d')
data['date'] = vtrans['date']
data = data.drop_duplicates(subset=["date"])

vtrans.set_index('EntranceTime')
data["date"] = pd.to_datetime(data["date"], format='%Y/%m/%d')
data["y"] = vtrans.groupby('date')['date'].transform('count')


data.head()

In [ ]:
data = data.set_index('date')
data = data.sort_index()
data = data.asfreq(freq='D', fill_value=np.nan)

print(f'Number of rows with missing values: {data.isnull().any(axis=1).mean()}')

In [ ]:
#!pip3 install sklearn
#!pip3 install matplotlib

import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Split into 75% training data and 25% testing
steps = 540
data_train = data[:-steps]
data_test  = data[-steps:]

fig, ax=plt.subplots(figsize=(12, 6))
data_train["y"].plot(ax=ax, label='train')
data_test["y"].plot(ax=ax, label='test')
ax.legend();




In [ ]:
#!pip3 install skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor


forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = 180
                )

forecaster.fit(y=data_train['y'])
forecaster

In [ ]:
predictions = forecaster.predict(steps=steps)
predictions.head()


In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
data_train["y"].plot(ax=ax, label='train')
data_test['y'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predictions')
ax.legend();